In [1]:
import numpy as np
import cvxpy as cp
from qsys import basis, mbasis, sigma_0, sigma_x, sigma_y, sigma_z
import qcvx

# Pauli matrices
I, X, Y, Z = sigma_0, sigma_x, sigma_y, sigma_z

In [2]:
'''The systems of the quantum time flip
      ____________________
      |                  |
0 ----|                  |---- 4
      |                  |
      |    2 -- C -- 3   |
 ctrl |                  |
1 ----|                  |---- 5
      |                  |
      ____________________
'''

# the dimension of system A
d = 2
# compute the Choi operator of QTF
dims = [d,2,d,d,d,2]
v = np.einsum(np.eye(d), [0,2], np.eye(d), [3,4], basis(2,0), [1], basis(2,0), [5]) + \
    np.einsum(np.eye(d), [0,3], np.eye(d), [2,4], basis(2,1), [1], basis(2,1), [5])
choi_qtf = np.einsum(v, range(6), v.conjugate(), range(6,12))

choi1 = choi_qtf.reshape(np.prod(dims), -1)
choi2 = np.einsum(choi_qtf, [0,1,2,3,4,5, 6,7,8,9,10,11], (I+X)/2, [1,7]).reshape(d**4*2, -1)
choi3 = np.einsum(choi_qtf, [0,1,2,3,4,5, 6,7,8,9,4,11], mbasis(d,0), [0,6], (I+X)/2, [1,7]).reshape(d**2*2, -1)

In [3]:
def robustness(choi_S, dims, precision):
    '''
    Calculate the robustness of input-output indefiniteness,
    which is an SDP problem.
    '''
    D = np.prod(dims)

    T = cp.Variable((D,D), hermitian=True)
    Sf = cp.Variable((D,D), hermitian=True)
    Sb = cp.Variable((D,D), hermitian=True)

    def ptrace(S, trace_list):
        return qcvx.partial_trace(S, dims, trace_list, discard=False)
    
    cons = [
        T >> 0,
        Sf >> 0,
        Sb >> 0,
        ptrace(T, [1,2,3]) == ptrace(T, [0,1,2,3]),
        ptrace(Sf, [1,2,3]) == ptrace(Sf, [0,1,2,3]),
        ptrace(Sb, [1,2,3]) == ptrace(Sb, [0,1,2,3]),
        choi_S + T == Sf + Sb,
        ptrace(Sf, [3]) == ptrace(Sf, [2,3]),
        ptrace(Sb, [3]) == ptrace(Sb, [1,3]),
        ptrace(T, [3]) == ptrace(T, [1,3]) + ptrace(T, [2,3]) - ptrace(T, [1,2,3]),
    ]

    prob = cp.Problem(cp.Minimize(cp.real(cp.trace(T) / (dims[0]*dims[2]))), cons)
    prob.solve(solver=cp.SCS, eps=precision)
    return prob.status, prob.value

precision = 1e-9
print(robustness(choi1, [2*d,d,d,2*d], precision), f'({precision})')
print(robustness(choi2, [d,d,d,2*d], precision), f'({precision})')
print(robustness(choi3, [1,d,d,2], precision), f'({precision})')

('optimal', 0.7777777777798457) (1e-09)
('optimal', 0.40068034300324606) (1e-09)
('optimal', 0.17157287526474022) (1e-09)
